### Install Dependencies

In [ ]:
!pip install opencv-python==3.4.2.16
!pip install imutils
!pip install scikit-image
!pip install matplotlib
!pip install numpy

### Import packages

In [1]:
import cv2
import numpy as np
import os
import imutils
from skimage.measure import compare_ssim
import matplotlib.pyplot as plt
%matplotlib inline

### Load yolo

In [ ]:
net=cv2.dnn.readNet("../Yolo/yolov3.weights","../Yolo/yolov3.cfg")
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_OPENCL)
layer_names = net.getLayerNames()
outputlayers = [layer_names[i[0]-1] for i in net.getUnconnectedOutLayers()]

with open("../Yolo/coco.names","r") as f:
    classes = [line.strip() for line in f.readlines()]

### Convert video to frames

In [ ]:
#change video name
cap = cv2.VideoCapture("./NMPS-CD/Road/Ref.mp4")

counter=1
while (cap.isOpened()):
    ret, frame = cap.read()
    if ret==0:
        break
    #cahnge path and file name
    FrameNo = './NMPS-CD/Road/Ref/Frame'+str(counter)+'.jpg'
    cv2.imwrite(FrameNo,frame)
    counter = counter + 1
cap.close()

### Load frames

In [ ]:
ref_dir = '../NMPS-CD/Road/Ref'
video_dir = '../NMPS-CD/Road/Track1'
#video_dir = '../NMPS-CD/Road/Track2'
ref_data = []
video_data = []
n_ref_frames=0
n_video_frames=0
i=1
for f1 in os.listdir(ref_dir):
    img = cv2.imread(os.path.join(ref_dir,'Frame'+str(i)+'.jpg'))
    img = cv2.resize(img, (720,1280), interpolation = cv2.INTER_AREA)
    ref_data.append(img)
    n_ref_frames+=1
    i+=1
i=1
for f1 in os.listdir(video_dir):
    img = cv2.imread(os.path.join(video_dir,'Frame'+str(i)+'.jpg'))
    img = cv2.resize(img, (720,1280), interpolation = cv2.INTER_AREA)
    video_data.append(img)
    n_video_frames+=1
    i+=1

### Find objects in each frame

In [ ]:
CONF_THRESHOLD = 0.5
NMS_THRESHOLD = 0.4
def post_process(frame, outs, conf_threshold, nms_threshold):
    frame_height = frame.shape[0]
    frame_width = frame.shape[1]

    # Scan through all the bounding boxes output from the network and keep only
    # the ones with high confidence scores. Assign the box's class label as the
    # class with the highest score.
    confidences = []
    boxes = []
    class_ids = []
    final_boxes = []
    final_confidence = []
    final_classes=[]
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > conf_threshold:
                center_x = int(detection[0] * frame_width)
                center_y = int(detection[1] * frame_height)
                width = int(detection[2] * frame_width)
                height = int(detection[3] * frame_height)
                left = int(center_x - width / 2)
                top = int(center_y - height / 2)
                confidences.append(float(confidence))
                boxes.append([left, top, width, height])
                class_ids.append(class_id)

    # Perform non maximum suppression to eliminate redundant
    # overlapping boxes with lower confidences.
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold,
                               nms_threshold)

    for i in indices:
        i = i[0]
        box = boxes[i]
        left = box[0]
        top = box[1]
        width = box[2]
        height = box[3]
        final_boxes.append(box)
        final_classes.append(str(classes[class_ids[i]]))
        #left, top, right, bottom = refined_box(left, top, width, height)
        #draw_predict(frame, confidences[i], left, top, left + width,
        #              top + height)
        #draw_predict(frame, confidences[i], left, top, right, bottom)
    return zip(final_boxes,  final_classes)

In [ ]:
IMG_WIDTH = 416
IMG_HEIGHT = 416
def yolo_get_objects(data):
    objs = []
    for image in data:
        # Create a 4D blob from a frame.
        blob = cv2.dnn.blobFromImage(image, 1 / 255, (IMG_WIDTH, IMG_HEIGHT),[0, 0, 0], 1, crop=False)

        # Sets the input to the network
        net.setInput(blob)

        # Runs the forward pass to get output of the output layers
        outs = net.forward(outputlayers)

        # Remove the bounding boxes with low confidence
        objs.append(list(post_process(image, outs, CONF_THRESHOLD, NMS_THRESHOLD)))
    return objs

ref_objs = yolo_get_objects(ref_data)
video_objs = yolo_get_objects(video_data)

### SIFT and change detection

In [ ]:
def getHomography(kpsA, kpsB, featuresA, featuresB, matches, reprojThresh):
    # convert the keypoints to numpy arrays
    kpsA = np.float32([kp.pt for kp in kpsA])
    kpsB = np.float32([kp.pt for kp in kpsB])
    
    if len(matches) > 4:

        # construct the two sets of points
        ptsA = np.float32([kpsA[m.queryIdx] for m in matches])
        ptsB = np.float32([kpsB[m.trainIdx] for m in matches])
        
        # estimate the homography between the sets of points
        (H, status) = cv2.findHomography(ptsA, ptsB, cv2.RANSAC,
            reprojThresh)
        return (matches, H, status)
    else:
        return None

In [ ]:
def remove_similar_objs(img1, img2, list1, list2):
    width = img1.shape[0]
    height = img1.shape[1]
    threshold = 10
    if len(list1) == 0 or len(list2) == 0:
        return (list1, list2)
    for obj1 in list1:
        left1, top1, width1, height1 = obj1[0]
        obj1_img = img1
        mask = np.zeros((width, height),np.uint8)
        mask = cv2.rectangle(mask, (left1, top1), (left1+width1, top1+height1), 1, thickness=-1)
        obj1_masked = cv2.bitwise_and(obj1_img, obj1_img, mask=mask)
        
        for obj2 in list2:
            left2, top2, width2, height2 = obj2[0]
            obj2_img = img2
            mask = np.zeros((width, height),np.uint8)
            mask = cv2.rectangle(mask, (left2, top2), (left2+width2, top2+height2), 1, thickness=-1)
            obj2_masked = cv2.bitwise_and(obj2_img, obj2_img, mask=mask)
            
            orb_detector = cv2.ORB_create(5000)
            kp1, d1 = orb_detector.detectAndCompute(obj1_masked, None) 
            kp2, d2 = orb_detector.detectAndCompute(obj2_masked, None)
            matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck = True)
            matches = matcher.match(d1, d2) 
            
            if(len(matches) >= threshold):
                list1.remove(obj1)
                list2.remove(obj2)
    return (list1, list2)

In [ ]:
def find_similar_objs(img1, img2, list1, list2):
    width = img1.shape[0]
    height = img1.shape[1]
    threshold = 20
    img1_list = []
    img2_list = []
    if len(list1) == 0 or len(list2) == 0:
        return (list1, list2)
    for obj1 in list1:
        left1, top1, width1, height1 = obj1[0]
        obj1_img = img1
        mask = np.zeros((width, height),np.uint8)
        mask = cv2.rectangle(mask, (left1, top1), (left1+width1, top1+height1), 1, thickness=-1)
        obj1_masked = cv2.bitwise_and(obj1_img, obj1_img, mask=mask)
        
        for obj2 in list2:
            left2, top2, width2, height2 = obj2[0]
            obj2_img = img2
            mask = np.zeros((width, height),np.uint8)
            mask = cv2.rectangle(mask, (left2, top2), (left2+width2, top2+height2), 1, thickness=-1)
            obj2_masked = cv2.bitwise_and(obj2_img, obj2_img, mask=mask)
            
            orb_detector = cv2.ORB_create(5000)
            kp1, d1 = orb_detector.detectAndCompute(obj1_masked, None) 
            kp2, d2 = orb_detector.detectAndCompute(obj2_masked, None)
            matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck = True)
            matches = matcher.match(d1, d2) 
            
            if(len(matches) >= threshold):
                img1_list.append(obj1)
                img2_list.append(obj2)
    return (img1_list, img1_list)

In [ ]:
def generate_obj_mask(shape, list):
    mask = np.ones(shape,np.uint8)
    for obj,label in list:
        left, top, width, height = obj
        mask = cv2.rectangle(mask, (left, top), (left+width, top+height), 0, thickness=-1)
    return mask

In [ ]:
def generate_inverted_obj_mask(shape, list):
    mask = np.ones(shape,np.uint8)
    for obj,label in list:
        left, top, width, height = obj
        mask = cv2.rectangle(mask, (left, top), (left+width, top+height), 0, thickness=-1)
    return mask

In [ ]:
ref_ptr = 0
for i in range(0,n_video_frames):
    # Open the image files. 
    img1_color = video_data[i]  # Image to be aligned. 
    img2_color = None    # Reference image. 
    matches=0
    kp1,d1=None,None
    kp2,d2=None,None
    while(True):
        img2_color=ref_data[ref_ptr]
        img1 = cv2.cvtColor(img1_color, cv2.COLOR_BGR2GRAY) 
        img2 = cv2.cvtColor(img2_color, cv2.COLOR_BGR2GRAY)
        img2_prev = None
        img2_fwd = None
        img1_prev_objs, img2_prev_objs = None, None
        img1_fwd_objs, img2_fwd_objs = None, None
        width, height = img2.shape
        ref_prev_ptr = ref_ptr
        ref_fwd_ptr = ref_ptr
        if(ref_ptr>1):
            ref_prev_ptr = ref_ptr-1
        if(ref_ptr<len(ref_data)-1):
            ref_fwd_ptr = ref_ptr+1
        
        
        img2_prev = cv2.cvtColor(ref_data[ref_prev_ptr], cv2.COLOR_BGR2GRAY)
        img2_fwd = cv2.cvtColor(ref_data[ref_fwd_ptr], cv2.COLOR_BGR2GRAY)
        
        img1_objs, img2_objs = remove_similar_objs(img1, img2, video_objs[i].copy(), ref_objs[ref_ptr].copy())
        img1_prev_objs, img2_prev_objs = remove_similar_objs(img1, img2_prev, video_objs[i].copy(), ref_objs[ref_prev_ptr].copy())
        img1_fwd_objs, img2_fwd_objs = remove_similar_objs(img1, img2_fwd, video_objs[i].copy(), ref_objs[ref_fwd_ptr].copy())
        
        orb_detector = cv2.ORB_create(5000) 
        orb_detector_prev = cv2.ORB_create(5000)
        orb_detector_fwd = cv2.ORB_create(5000)

        kp1, d1 = orb_detector.detectAndCompute(img1, generate_obj_mask((width,height), img1_objs)) 
        kp2, d2 = orb_detector.detectAndCompute(img2, generate_obj_mask((width,height), img2_objs)) 
        kp3, d3 = orb_detector_prev.detectAndCompute(img1, generate_obj_mask((width,height), img1_prev_objs))
        kp4, d4 = orb_detector_prev.detectAndCompute(img2_prev, generate_obj_mask((width,height), img2_prev_objs))
        kp5, d5 = orb_detector.detectAndCompute(img1, generate_obj_mask((width,height), img1_fwd_objs)) 
        kp6, d6 = orb_detector.detectAndCompute(img2_fwd, generate_obj_mask((width,height), img2_fwd_objs))
 
        matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck = True) 

        matches = matcher.match(d1, d2) 
        matches_prev = matcher.match(d3, d4) 
        matches_fwd=matcher.match(d5, d6) 
        
        if(max(len(matches),len(matches_prev),len(matches_fwd)) == len(matches)):
            break
        elif(max(len(matches),len(matches_prev),len(matches_fwd)) == len(matches_prev)):
            ref_ptr-=2
            continue
        else:
            ref_ptr+=2
            continue

    matches.sort(key = lambda x: x.distance) 
 
    matches = matches[:int(len(matches)*90)] 
    no_of_matches = len(matches) 

    p1 = np.zeros((no_of_matches, 2)) 
    p2 = np.zeros((no_of_matches, 2)) 

    for i in range(len(matches)): 
        p1[i, :] = kp1[matches[i].queryIdx].pt 
        p2[i, :] = kp2[matches[i].trainIdx].pt 

    homography, mask = cv2.findHomography(p1, p2, cv2.RANSAC) 

    transformed_img = cv2.warpPerspective(img1_color, 
                        homography, (height, width)) 

    (score, diff) = compare_ssim(img2,cv2.cvtColor(transformed_img, cv2.COLOR_BGR2GRAY), full=True)
    diff = (diff * 255).astype("uint8")
                                                             
    thresh = cv2.threshold(diff, 0, 255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    
    result_img = np.concatenate((np.concatenate((img1,img2),axis=1), np.concatenate((diff,thresh),axis=1)), axis=0)
                                                             
    cv2.namedWindow("output", cv2.WINDOW_NORMAL)
    cv2.imshow("output", result_img)
    cv2.imwrite('../NMPS-CD/Road/Output-Track1/'+str(i)+'.jpg',result_img)
    
    keyboard = cv2.waitKey(30) & 0xFF
    if keyboard == 'q' or keyboard == 27:
        break
    
cv2.destroyAllWindows()

In [ ]:
cv2.namedWindow("thresh", cv2.WINDOW_NORMAL)
cv2.imshow("thresh", ref_data[0])
keyboard = cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
mask = genereate_obj_mask((1280,720),)
img1 = cv2.cvtColor(video_data[0], cv2.COLOR_BGR2GRAY) 
obj1_masked = cv2.bitwise_and(img1, img1, mask=mask)
cv2.namedWindow("mask1", cv2.WINDOW_NORMAL)
cv2.imshow("mask1", obj1_masked)
cv2.namedWindow("mask", cv2.WINDOW_NORMAL)
cv2.imshow("mask", mask)
keyboard = cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
def find_new_objs(frames, objs):
    new_objs = []
    history = []
    for i in range(1,len(frames),1):
        img1_objs, img2_objs = remove_similar_objs(frames[i-1], frames[i], objs[i-1].copy(), objs[i].copy())
        for obj in img2_objs:
            new_objs.append((i,obj))
    return new_objs

In [ ]:
ref_new_objs = find_new_objs(ref_data, ref_objs)

In [ ]:
ref_new_objs

In [ ]:
import time
for frame in ref_data:
    cv2.namedWindow("mask", cv2.WINDOW_NORMAL)
    cv2.imshow("mask", frame)
    time.sleep(0.3)
    keyboard = cv2.waitKey(30) & 0xFF
    if keyboard == 'q' or keyboard == 27:
        break
cv2.destroyAllWindows()

In [ ]:
cv2.namedWindow("mask", cv2.WINDOW_NORMAL)
cv2.imshow("mask", ref_data[84])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
from scipy.spatial import distance as dist
from collections import OrderedDict
import numpy as np

class CentroidTracker():
    def __init__(self, maxDisappeared=30):
        self.nextObjectID = 0
        self.objects = OrderedDict()
        self.disappeared = OrderedDict()
        self.maxDisappeared = maxDisappeared
        self.appearing = []
        self.disappearing = []
        self.frameNo = 0

    def register(self, centroid):
        self.objects[self.nextObjectID] = centroid
        self.disappeared[self.nextObjectID] = 0
        self.appearing.append((frameNo, self.nextObjectID, centroid))
        self.nextObjectID += 1

    def deregister(self, objectID):
        self.disappearing.append((frameNo, self.nextObjectID,self.objects[objectID]))
        del self.objects[objectID]
        del self.disappeared[objectID]

    def update(self, rects):
        if len(rects) == 0:
            for objectID in list(self.disappeared.keys()):
                self.disappeared[objectID] += 1
                if self.disappeared[objectID] > self.maxDisappeared:
                    self.deregister(objectID)
            return self.objects
        inputCentroids = np.zeros((len(rects), 2), dtype="int")
        for (i, (startX, startY, endX, endY)) in enumerate(rects):
            cX = int((2*startX + endX) / 2.0)
            cY = int((2*startY + endY) / 2.0)
            inputCentroids[i] = (cX, cY)
        if len(self.objects) == 0:
            for i in range(0, len(inputCentroids)):
                self.register(inputCentroids[i])
        else:
            objectIDs = list(self.objects.keys())
            objectCentroids = list(self.objects.values())
            D = dist.cdist(np.array(objectCentroids), inputCentroids)
            rows = D.min(axis=1).argsort()
            cols = D.argmin(axis=1)[rows]
            usedRows = set()
            usedCols = set()
            for (row, col) in zip(rows, cols):
                if row in usedRows or col in usedCols:
                    continue
                objectID = objectIDs[row]
                self.objects[objectID] = inputCentroids[col]
                self.disappeared[objectID] = 0
                usedRows.add(row)
                usedCols.add(col)
            unusedRows = set(range(0, D.shape[0])).difference(usedRows)
            unusedCols = set(range(0, D.shape[1])).difference(usedCols)
            if D.shape[0] >= D.shape[1]:
                for row in unusedRows:
                    objectID = objectIDs[row]
                    self.disappeared[objectID] += 1
                    if self.disappeared[objectID] > self.maxDisappeared:
                        self.deregister(objectID)
            else:
                for col in unusedCols:
                    self.register(inputCentroids[col])
        frameNo+=1
        return self.objects

In [ ]:
ct = CentroidTracker()
(H, W) = (None, None)

for i in range(0,len(ref_data)):
    objects = ct.update([box[0] for box in ref_objs[i]])
    #frame = ref_data[i].copy()
    #for obj in ref_objs[i]:
    #    left, top, width, height = obj[0]
    #    cv2.rectangle(frame, (left, top), (left + width,top + height),(0, 255, 0),2)
    if W is None or H is None:
        (H, W) = frame.shape[:2]
    # loop over the tracked objects
    for (objectID, centroid) in objects.items():
        # draw both the ID of the object and the centroid of the
        # object on the output frame
        text = "ID {}".format(objectID)
        cv2.putText(frame, text, (centroid[0] - 10, centroid[1] - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        cv2.circle(frame, (centroid[0], centroid[1]), 4, (0, 255, 0), -1)

    # show the output frame
    cv2.namedWindow("Frame", cv2.WINDOW_NORMAL);cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

# do a bit of cleanup
cv2.destroyAllWindows()

In [ ]:
ref_objs[i]

In [ ]:
from skimage.measure import compare_ssim as ssim
def mse(imageA, imageB):
    err = np.sum((imageA.astype("float")-imageB.astype("float"))**2)
    err /= float(imageA.shape[0] * imageA.shape[1])
    return err
def compare_image(imageA,imageB):
    m = mse(imageA,imageB)
    s = ssim(imageA,imageB)
    fig = plt.figure("abc")
    plt.suptitle("MSE: %.2f, SSIM: %.2f" % (m,s))
    
    ax = fig.add_subplot(1,2,1)
    plt.imshow(imageA, cmap = plt.cm.gray)
    plt.axis("off")
    
    ax = fig.add_subplot(1,2,2)
    plt.imshow(imageB, cmap = plt.cm.gray)
    plt.axis("off")
    return (mse, ssim)